In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import folium
import requests
import logging
from folium.plugins import TimestampedGeoJson, HeatMap

pd.options.display.max_colwidth = 200

In [2]:
file_name = 'simulation_log_large.csv.gz'

data = pd.read_csv(file_name, sep=';',
                   # convert column to dict
                   converters={'details': lambda v: eval(v)})

print(f'Data shape: {data.shape}')

data.head()

Data shape: (49999, 9)


,clock_time,object_type,uuid,itinerary_id,from_state,to_state,lon,lat,details
0,0,vehicle,5a39815303224fb8868a6c74aa148ca8,NaN,offline,idling,-73.997155,40.711960,{}
1,0,vehicle,967613f781a74375b197161a424d189c,NaN,offline,idling,-73.982318,40.769531,{}
2,0,vehicle,ba35d3cdaa4844329e24e713b4f9a3d3,NaN,offline,idling,-73.982430,40.785004,{}
3,0,vehicle,6e291567fe884ce388fca8423b9ab1c1,NaN,offline,idling,-74.007424,40.726010,{}
4,0,vehicle,cf97a35b325e4afdbb26ff4585506a2f,NaN,offline,idling,-73.978013,40.758114,{}


In [3]:
# convert "details" column into a separate DataFrame
details = data.details.apply(pd.Series)
details[~data.itinerary_id.isna()].head()

,dropoff,eta,pickup,dst,route_duration,route_distance,trip_duration,trip_distance,vid,stop
201,NaN,9.0,76a7abf718f640b0a4acd90f832170d8,"{'lat': 40.732983, 'lon': -74.006462}",6.0,0.455,0.0,0.0,NaN,NaN
202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0eccbb469a424009b061b7f80f60f878,NaN
203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0eccbb469a424009b061b7f80f60f878,NaN
205,NaN,5.0,3a7e5735296a4f3cbe8fb4b6bc0ba52c,"{'lat': 40.758106, 'lon': -73.992729}",3.0,0.279,0.0,0.0,NaN,NaN
206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3e9db6bb80f44fa2bed0a0cf8d1c5e7c,NaN


## Pickup locations

In [4]:
idx = (data.object_type == 'booking') & (data.from_state == 'pickup')
points = data[idx][['lat', 'lon']]

m = folium.Map(location=[40.76953, -73.98232], zoom_start=12)

HeatMap(points, radius=15, min_opacity=0.2).add_to(m)

m

### Vehicle movements

**WARNING**: requires [OSRM]('http://localhost:5010/route/v1/driving/-73.977547,40.763336;-73.938263,40.791920?geometries=geojson') running localy on port 5010. 

In [5]:
from requests.exceptions import ConnectionError
    
OSRM_HOST = 'localhost'
OSRM_PORT = 5010
OSRM_URL_FORMAT = 'http://{osrm_host}:{osrm_port}/route/v1/driving/{coords}?geometries=geojson'

def get_vehicle_route(from_point, to_point):
    """lon/lat order"""
    
    geoms = []
    
    try:
        coords = f'{from_point[0]},{from_point[1]};{to_point[0]},{to_point[1]}'
        url = OSRM_URL_FORMAT.format(
            osrm_host=OSRM_HOST, 
            osrm_port=OSRM_PORT, 
            coords=coords)

        # TODO: check response code
        osrm_response = requests.get(url)
        
        for route in osrm_response.json()['routes']:
            geom = {
                'type': 'Feature',
                'geometry': route['geometry'],
                'properties': {}
            }
            
            geoms.append(geom)
            
        return geoms
    except ConnectionError as e:
        logging.warning(e)
        
        geom = {
            'type': 'Feature',
            'geometry': {
                'type': 'LineString',
                'coordinates': [list(from_point), list(to_point)]
            },
            'properties': {}
        }
    return geoms

In [6]:
m = folium.Map(location=[40.76953, -73.98232], zoom_start=12)

def style_function(feature):
    colors = ['red', 'blue', 'brown', 'yellow', 'green']
    idx = feature['properties']['vehicle_index']
    if idx >= len(colors):
        idx = -1
    
    return {
        'opacity': 0.5, 
        'weight': 2,
        'color': colors[idx]
    }

num_vehicles = 0
total_vehicles = 3

all_routes = []
now = datetime.now()

def calc_time_steps(from_time, to_time, steps):
    start = from_time
    delta = round((to_time - from_time).seconds / 60 / steps)
    
    times = [str(from_time)]
    for i in range(1, steps - 1):
        start += timedelta(minutes=delta)
        times.append(str(start))
    times.append(str(to_time))
    
    return times

idx = (data.object_type == 'vehicle')

for i, (grp, vehicle_changes) in enumerate(data[idx].groupby('uuid')):
    for grp, items in vehicle_changes.groupby('itinerary_id'):
        line = []
        time_steps = []
        for item in items.itertuples():
            line.append([item.lon, item.lat])
            
            tm = now + timedelta(minutes=(item.clock_time))
            time_steps.append(tm)

        # only 2 points - from and to
        if len(line) != 4:
            continue
            
        for from_point, to_point in zip(line[::2], line[1::2]):
            
            for geom in get_vehicle_route(from_point, to_point):
                start_at = time_steps.pop(0)
                stop_at = time_steps.pop(0)

                num_points = len(geom['geometry']['coordinates'])
                geom['properties']['times'] = calc_time_steps(start_at, stop_at, num_points)
                geom['properties']['vehicle_index'] = i
                geom['properties']['style'] = style_function(geom)
                
                all_routes.append(geom)
                
                folium.GeoJson(
                    geom,
                    name='geojson',
                    style_function=style_function
                ).add_to(m)
                
    num_vehicles += 1
    if num_vehicles == total_vehicles:
        break
            
m

### Time stamped moves

In [7]:
m = folium.Map(location=[40.76953, -73.98232], zoom_start=12)

TimestampedGeoJson(
    all_routes,
    period='PT10M',
    add_last_point=False,
    auto_play=False, 
    loop=False, 
    max_speed=1,
    loop_button=True,
    time_slider_drag_update=True
).add_to(m)

m